In [1]:
import transformers
print(transformers.__version__)


4.49.0


In [ ]:
%pip install langchain langchain-ollama ollama

In [ ]:
from langchain_ollama import OllamaLLM

llama_model = OllamaLLM(model = "llama3.3")
for chunk in llama_model.stream("generate a nda contract "):
    print(chunk , end = "", flush = True)



In [1]:
from langchain_ollama import OllamaLLM
from typing import List
import re

class ContinuousContractSummarizer:
    def __init__(self, model: str = "llama3.3"):
        self.llm = OllamaLLM(model=model)

    def _split_contract(self, text: str) -> List[str]:
        """Smart splitting prioritizing legal headings"""
        sections = re.split(r'\n\s*(ARTICLE|SECTION)\s+[IVXLCDM]+\s*\n', text)
        if len(sections) > 1:
            return sections
        
        return re.split(r'\n\d+\.\s+[A-Z][A-Z\s]+\n', text) or [text]

    def _generate_prompt(self, text: str, is_final: bool = False) -> str:
        if is_final:
            return f"""
            Create a FLOWING CONTRACT SUMMARY with these requirements:
            
            1. PRESERVE EXACTLY:
               - Numbers ($10,000) and dates (2025-01-01)
               - Key obligations (shall/must/will)
               - Conditional terms (if/unless)
            
            2. ORGANIZE NATURALLY by:
               - Who is involved
               - What must be done
               - When payments occur
               - How to terminate
            
            3. WRITE IN CONTINUOUS PROSE that reads like a narrative.
            
            CONTENT TO SUMMARIZE:
            {text}


            - Exclude explantion
            """
        else:
            return f"""
            Extract CRUCIAL CONTRACT TERMS for later summarization:
            
            RULES:
            - include parties details
            - keep termination, financial and risks
            - Copy VERBATIM: amounts, dates, deadlines
            - Keep obligations and conditions intact
            - Ignore boilerplate and signatures
            
            TEXT:
            {text}
            """

    def summarize(self, contract_text: str):
        chunks = self._split_contract(contract_text)
        
        extracted_terms = []
        for chunk in chunks:
            prompt = self._generate_prompt(chunk)
            for chunk in self.llm.stream(prompt):
                print(chunk, end="", flush=True)
                extracted_terms.append(chunk)
        
        combined_terms = "\n".join(extracted_terms)
        final_prompt = self._generate_prompt(combined_terms, is_final=True)
        
        print("\n\n================ FINAL SUMMARY ==================")
        for chunk in self.llm.stream(final_prompt):
            print(chunk, end="", flush=True)

In [ ]:

summarizer = ContinuousContractSummarizer()
contract_text = input("Paste contract text: ")
summarizer.summarize(contract_text)